In [34]:
# %matplotlib inline
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
from pandas_profiling import ProfileReport
from scipy.stats import pearsonr
from scipy import stats
from statsmodels.stats.anova import AnovaRM
from sklearn.covariance import GraphicalLasso, graphical_lasso,EmpiricalCovariance
from sklearn.decomposition import PCA, FastICA
from nilearn.maskers import NiftiMasker
from nilearn.maskers import NiftiLabelsMasker
from nilearn.maskers import NiftiMapsMasker
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from nilearn.plotting import plot_stat_map, show, plot_epi, plot_prob_atlas, find_xyz_cut_coords, plot_roi
from nilearn import image
from nilearn.plotting import plot_stat_map, show
from nilearn import plotting
import seaborn as sns
import os
import importlib

import gi_gmn
import gi_gmn.utils as giu
import gi_gmn.models as gim

In [79]:
NP_train = 19
NC_train = 25
NP = 27
NC = 36
NPRE = 16
NP_test = NP - NP_train
NC_test = NC - NC_train
TR_P = 122
TR_C = 170
NRecord = 2
ROI = 120

path = "/users/grad/fmoradi/Graphical_LASSO/ADHD_WashU/data/"
masker = NiftiLabelsMasker(labels_img="AAL_gm.nii", resampling_target = "data",standardize=True)

#based on AAL atlas
all_col = ['Precentral_L', 'Precentral_R', 'Frontal_Sup_2_L', 'Frontal_Sup_2_R', 'Frontal_Mid_2_L', 'Frontal_Mid_2_R', 'Frontal_Inf_Oper_L', 'Frontal_Inf_Oper_R', 'Frontal_Inf_Tri_L', 'Frontal_Inf_Tri_R', 'Frontal_Inf_Orb_2_L', 'Frontal_Inf_Orb_2_R', 'Rolandic_Oper_L', 'Rolandic_Oper_R', 'Supp_Motor_Area_L', 'Supp_Motor_Area_R', 'Olfactory_L', 'Olfactory_R', 'Frontal_Sup_Medial_L', 'Frontal_Sup_Medial_R', 'Frontal_Med_Orb_L', 'Frontal_Med_Orb_R', 'Rectus_L', 'Rectus_R', 'OFCmed_L', 'OFCmed_R', 'OFCant_L', 'OFCant_R', 'OFCpost_L', 'OFCpost_R', 'OFClat_L', 'OFClat_R', 'Insula_L', 'Insula_R', 'Cingulate_Ant_L', 'Cingulate_Ant_R', 'Cingulate_Mid_L', 'Cingulate_Mid_R', 'Cingulate_Post_L', 'Cingulate_Post_R', 'Hippocampus_L', 'Hippocampus_R', 'ParaHippocampal_L', 'ParaHippocampal_R', 'Amygdala_L', 'Amygdala_R', 'Calcarine_L', 'Calcarine_R', 'Cuneus_L', 'Cuneus_R', 'Lingual_L', 'Lingual_R', 'Occipital_Sup_L', 'Occipital_Sup_R', 'Occipital_Mid_L', 'Occipital_Mid_R', 'Occipital_Inf_L', 'Occipital_Inf_R', 'Fusiform_L', 'Fusiform_R', 'Postcentral_L', 'Postcentral_R', 'Parietal_Sup_L', 'Parietal_Sup_R', 'Parietal_Inf_L', 'Parietal_Inf_R', 'SupraMarginal_L', 'SupraMarginal_R', 'Angular_L', 'Angular_R', 'Precuneus_L', 'Precuneus_R', 'Paracentral_Lobule_L', 'Paracentral_Lobule_R', 'Caudate_L', 'Caudate_R', 'Putamen_L', 'Putamen_R', 'Pallidum_L', 'Pallidum_R', 'Thalamus_L', 'Thalamus_R', 'Heschl_L', 'Heschl_R', 'Temporal_Sup_L', 'Temporal_Sup_R', 'Temporal_Pole_Sup_L', 'Temporal_Pole_Sup_R', 'Temporal_Mid_L', 'Temporal_Mid_R', 'Temporal_Pole_Mid_L', 'Temporal_Pole_Mid_R', 'Temporal_Inf_L', 'Temporal_Inf_R', 'Cerebelum_Crus1_L', 'Cerebelum_Crus1_R', 'Cerebelum_Crus2_L', 'Cerebelum_Crus2_R', 'Cerebelum_3_L', 'Cerebelum_3_R', 'Cerebelum_4_5_L', 'Cerebelum_4_5_R', 'Cerebelum_6_L', 'Cerebelum_6_R', 'Cerebelum_7b_L', 'Cerebelum_7b_R', 'Cerebelum_8_L', 'Cerebelum_8_R', 'Cerebelum_9_L', 'Cerebelum_9_R', 'Cerebelum_10_L', 'Cerebelum_10_R', 'Vermis_1_2', 'Vermis_3', 'Vermis_4_5', 'Vermis_6', 'Vermis_7', 'Vermis_8', 'Vermis_9', 'Vermis_10']


In [72]:
#reading the data
#only considering the study3 which is 46 subjects


all_patients = pd.DataFrame()

files = os.listdir(path)
for file in files:
    if file != '.DS_Store':
        
        print(file)
        # t = data_path+"/"+file + "/" + "ses-pre/func/" + file +"_ses-pre_task-rest_run-01_bold.denoise/Denoised_"+ file +"_ses-pre_task-rest_run-01_bold_MNI.nii.gz"

        data_file = path +file+ "/ses-1/func/" 
        # print(data_file)

        files_ = os.listdir(data_file)
        records = []

        for sub_file in files_:
            if sub_file.endswith('.denoise') and sub_file.count('reststudy3') == 1 and file!= "sub-0015052" :

                print(sub_file)
                records.append(sub_file.split(".")[0])
        if file != "sub-0015052":

            patient_record1_dir = data_file + records[0] +  ".denoise/Denoised_"+ records[0]+ "_MNI.nii.gz"
            patient_record2_dir = data_file + records[1] +  ".denoise/Denoised_"+ records[1]+ "_MNI.nii.gz"

            gr1 = giu.get_subject(patient_record1_dir,patient_record2_dir,masker,TR_P, NRecord, ROI)
            all_patients = all_patients.append(pd.DataFrame(gr1),ignore_index=True)


# all_patients.to_csv('../data/ADHD_WashU/patient_washU_study3_tworecords.csv')

sub-0015033
sub-0015033_ses-1_task-reststudy3_run-2_bold.denoise
sub-0015033_ses-1_task-reststudy3_run-1_bold.denoise
(91, 109, 91, 122)
(91, 109, 91, 244) (902629, 244)
sub-0015039
sub-0015039_ses-1_task-reststudy3_run-2_bold.denoise
sub-0015039_ses-1_task-reststudy3_run-1_bold.denoise
(91, 109, 91, 122)
(91, 109, 91, 244) (902629, 244)
sub-0015043
sub-0015043_ses-1_task-reststudy3_run-1_bold.denoise
sub-0015043_ses-1_task-reststudy3_run-2_bold.denoise
(91, 109, 91, 122)
(91, 109, 91, 244) (902629, 244)
sub-0015034
sub-0015034_ses-1_task-reststudy3_run-2_bold.denoise
sub-0015034_ses-1_task-reststudy3_run-1_bold.denoise
(91, 109, 91, 122)
(91, 109, 91, 244) (902629, 244)
sub-0015049
sub-0015049_ses-1_task-reststudy3_run-2_bold.denoise
sub-0015049_ses-1_task-reststudy3_run-1_bold.denoise
(91, 109, 91, 122)
(91, 109, 91, 244) (902629, 244)
sub-0015050
sub-0015050_ses-1_task-reststudy3_run-2_bold.denoise
sub-0015050_ses-1_task-reststudy3_run-1_bold.denoise
(91, 109, 91, 122)
(91, 109, 91,

In [77]:

all_control  = pd.read_csv('../../Ataxia/control_data_twoyear.csv',index_col=0).to_numpy()
all_patient  = pd.read_csv('../data/ADHD_WashU/patient_washU_study3_tworecords.csv',index_col=0).to_numpy()


In [81]:
all_patients.shape, all_control.shape

((6588, 120), (12240, 120))

In [83]:
importlib.reload(gi_gmn)
importlib.reload(gi_gmn.utils)
importlib.reload(gi_gmn.models)

# group level GMN - performance on test and saving preciison matrix 

alpha_p = 0.1
alpha_c = 0.1

model = gim.GroupSubjectGMN(
            NC, 
            NP, 
            NC_test, 
            TR_P,
            TR_C, 
            NP_test, 
            NRecord, 
            ROI
)
precision_p, precision_c = model.group_level(all_patients, all_control, alpha_p, alpha_c)
precision_c.shape




Model: Group-level GMN
0.825 0.1274754878398196 0.9818181818181818 0.036363636363636376


(120, 120)

In [ ]:
thr_weak = 0.1
Thresholding_diff = 0.12

copy_norm_p, copy_norm_c = model.prune_connections(precision_p, precision_c, thr_weak, Thresholding_diff)

giu.degree_sparsity(copy_norm_c), giu.degree_sparsity(copy_norm_p)

df_CONN, regions, connU = model.extract_connections_regions(copy_norm_p, copy_norm_c, all_col)